# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

Using TensorFlow backend.


## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [2]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Create Model and Load Trained Weights

In [3]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [4]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']


## Run Object Detection

In [ ]:
# Load a random image from the images folder
# file_names = next(os.walk(IMAGE_DIR))[2]
# image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
image = skimage.io.imread('/Users/aaron/Desktop/LeoEast/LeoEast0245.JPG')

# Run detection
results = model.detect([image], verbose=1)

# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])

## Count features and add to pandas dataframe

In [ ]:
import pandas as pd
import glob, time

jpgs = sorted(glob.glob(os.path.join('/Users/aaron/Desktop/LeoEast', '*.JPG')))

counter = 1
data = []
for jpg in jpgs:
    start = time.time()
    image = skimage.io.imread(jpg)
    
    # Run detection
    results = model.detect([image], verbose=0) # switch to verbose = 1 for image info
    
    # Count features
    vals = [item for item in list(results[0].get('class_ids')) if str(item) in ['1','9']]
    print('There are {0} watercraft and {1} boat anglers'.format(vals.count(9), vals.count(1)))

    # Visualize results
    #r = results[0]
    #visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
    #                            class_names, r['scores'])
    
    # Add results to pandas dataframe
    data.append({'image_path':os.path.basename(jpg), 'watercraft':vals.count(9), 'boat_anglers':vals.count(1)})
    
    # Keep a verbose tally on where we are at on the processing
    end = time.time()
    print('{0}: {1} of {2} processed in {3} seconds'.format(os.path.basename(jpg), counter, len(jpgs), round(end - start, 2)))
    counter += 1
    
df = pd.DataFrame(data, columns = ['image_path', 'watercraft', 'boat_anglers'])
df
print("Processing complete")


There are 0 watercraft and 0 boat anglers
LeoEast0001.JPG: 1 of 2516 processed in 12.66 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0002.JPG: 2 of 2516 processed in 12.48 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0003.JPG: 3 of 2516 processed in 13.75 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0004.JPG: 4 of 2516 processed in 13.49 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0005.JPG: 5 of 2516 processed in 13.55 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0006.JPG: 6 of 2516 processed in 13.65 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0007.JPG: 7 of 2516 processed in 12.89 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0008.JPG: 8 of 2516 processed in 12.9 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0009.JPG: 9 of 2516 processed in 12.88 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0010.JPG: 10 of 2516 processed in 12.93 seconds
There are 0 watercraft and 0 boat angler

There are 1 watercraft and 1 boat anglers
LeoEast0086.JPG: 86 of 2516 processed in 12.88 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0087.JPG: 87 of 2516 processed in 12.86 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0088.JPG: 88 of 2516 processed in 12.49 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0089.JPG: 89 of 2516 processed in 12.06 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0090.JPG: 90 of 2516 processed in 12.36 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0091.JPG: 91 of 2516 processed in 12.27 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0092.JPG: 92 of 2516 processed in 12.38 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0093.JPG: 93 of 2516 processed in 12.51 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0094.JPG: 94 of 2516 processed in 12.37 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0095.JPG: 95 of 2516 processed in 14.64 seconds
There are 0 watercraft and 0 b

There are 0 watercraft and 0 boat anglers
LeoEast0170.JPG: 170 of 2516 processed in 12.34 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0171.JPG: 171 of 2516 processed in 13.09 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0172.JPG: 172 of 2516 processed in 12.27 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0173.JPG: 173 of 2516 processed in 14.2 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0174.JPG: 174 of 2516 processed in 12.83 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0175.JPG: 175 of 2516 processed in 12.75 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0176.JPG: 176 of 2516 processed in 12.7 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0177.JPG: 177 of 2516 processed in 12.6 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0178.JPG: 178 of 2516 processed in 13.31 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0179.JPG: 179 of 2516 processed in 11.95 seconds
There are 0 watercraft 

There are 0 watercraft and 0 boat anglers
LeoEast0254.JPG: 254 of 2516 processed in 12.55 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0255.JPG: 255 of 2516 processed in 12.2 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0256.JPG: 256 of 2516 processed in 13.14 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0257.JPG: 257 of 2516 processed in 13.1 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0258.JPG: 258 of 2516 processed in 13.19 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0259.JPG: 259 of 2516 processed in 13.17 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0260.JPG: 260 of 2516 processed in 12.89 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0261.JPG: 261 of 2516 processed in 12.12 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0262.JPG: 262 of 2516 processed in 12.58 seconds
There are 0 watercraft and 0 boat anglers
LeoEast0263.JPG: 263 of 2516 processed in 12.66 seconds
There are 0 watercraft

'L__00009.JPG'